<a href="https://colab.research.google.com/github/atharvadesai1/ChatApp/blob/main/lungs_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/"

In [ ]:
import zipfile
zip_path = "/content/drive/My Drive/Lung_Disease_Dataset.zip"
extract_path = "/content/Lung_Disease_Dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Complete!")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
train_path = '/content/Lung_Disease_Dataset/train'
valid_path = '/content/Lung_Disease_Dataset/val'
test_path = '/content/Lung_Disease_Dataset/test'

In [ ]:
# Define class labels
class_labels = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

In [ ]:
# Image preprocessing & augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Load training and validation datasets
train_gen = datagen.flow_from_directory(
    train_path, target_size=(380, 380), batch_size=32, class_mode='categorical')

valid_gen = datagen.flow_from_directory(
    valid_path, target_size=(380, 380), batch_size=32, class_mode='categorical')

In [ ]:
# Load EfficientNet with pre-trained weights and remove top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(380, 380, 3))
base_model.trainable = False  # Freeze base model

In [ ]:
# Build model
my_model = Sequential([
    Input(shape=(380, 380, 3)),
    base_model,
    GlobalMaxPooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

In [ ]:
# Compile model
my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
epochs = 10  # Adjust based on available resources
history = my_model.fit(train_gen, validation_data=valid_gen, epochs=epochs)

In [ ]:
# Save the trained model
my_model.save("lungs_model.h5")
print("Model saved as lungs_model.h5")

In [ ]:
# Plot training history
def plot_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')
    plt.show()

plot_history(history)

In [ ]:
# Function to predict an image
def predict_image(image_path, model, class_labels):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (380, 380))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    class_name = class_labels[class_idx]
    confidence = prediction[0][class_idx]

    plt.imshow(cv2.imread(image_path))
    plt.axis('off')
    plt.title(f'Predicted: {class_name} ({confidence:.2f})')
    plt.show()

    return class_name

In [ ]:
# Example usage: Upload an image and predict
def upload_and_predict():
    from google.colab import files
    uploaded = files.upload()
    for img_path in uploaded.keys():
        print(f"Processing: {img_path}")
        prediction = predict_image(img_path, my_model, class_labels)
        print(f"Predicted Disease: {prediction}")

upload_and_predict()